In [2]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
import numpy as np
from typing import List, Union
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, assemble
from qiskit.compiler import transpile
from qiskit.circuit import Gate
from qiskit.circuit.library.standard_gates import *
from qiskit.circuit.library import QFT

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from validation_challenge4_4c import prepare_ex4c, grade
from visualize_histogram import plot_hist, flag_correct, add_correct, both_correct, reinit_correct
from problem_set import validation_set_4c_fixed, validation_set_4c_random

myps = validation_set_4c_fixed + validation_set_4c_random

In [30]:
def solver_function(L1: list, L2: list, C1: list, C2: list,
                    C_max: int) -> QuantumCircuit:
    dotranspile = True

    op_lvl = 2
    bgate = ['cx', 'rz', 'sx']
    # the number of qubits representing answers
    index_qubits = len(L1)

    # the maximum possible total cost
    max_c = sum([max(l0, l1) for l0, l1 in zip(C1, C2)])
    min_c = sum([min(l0, l1) for l0, l1 in zip(C1, C2)])

    adjust = 1  # Let == C_max through, don't seem to need
    target_min_c = 1  #make min_c 1

    # adjust to make C_max 2**w
    w_bits = math.ceil(math.log(C_max - min_c + target_min_c, 2))
    if 2**w_bits == C_max - min_c + target_min_c:
        target_min_c -= 1 # make sure I can downshift the shift on actual costs by 1
        # this will let == C_max values through
    diff = 2**(w_bits) - (C_max + adjust - min_c + target_min_c)

    #assuming C_max <= max_c otherwise problem is meaningless
    max_c = max_c - min_c + target_min_c + diff  #reduce number of qubits used
    # print(
    #     'after adjust C_max = 2**w_bits, min_c, max_c, shift for C_max, shift for minmax 1 less'
    # )
    # print(2**w_bits, target_min_c, max_c, 2**w_bits - C_max,
    #       min_c - target_min_c - diff)
    data_qubits = w_bits + 1
    ultra = True
    if max_c > 2**(data_qubits) - 1:
        ultra = False
        data_qubits = math.ceil(math.log(max_c, 2))
        if max_c > 2**(data_qubits) - 1:
            data_qubits += 1
    ### Phase Operator ###
    # return part
    def phase_return(index_qubits: int,
                     gamma: float,
                     L1: list,
                     L2: list,
                     to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(index_qubits, "index")
        qc = QuantumCircuit(qr_index)

        ##############################
        ### U_1(gamma * (lambda2 - lambda1)) for each qubit ###
        # Provide your code here

        for ii in range(index_qubits):
            # if problem statement is correct,
            # L2 > L1 cannot give zero here
            qc.p((-gamma * (L2[ii] - L1[ii])/2) % (2 * np.pi), qr_index[ii])

        if dotranspile:
            qc = transpile(qc,
                           optimization_level=op_lvl,
                           basis_gates=bgate,
                           seed_transpiler=42)
        ##############################
        return qc.to_gate(label=" phase return ") if to_gate else qc

    # penalty part
    def subroutine_add_const(data_qubits: int,
                             const: int,
                             to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(1)
        qr_data = QuantumRegister(data_qubits)
        qc = QuantumCircuit(qr_index, qr_data)
        ##############################
        ### Phase Rotation ###
        # Provide your code here
        for qb in range(data_qubits):
            twoP = 2**(qb + 1)  #twoP*2
            c = const % twoP
            if c != 0:
                phase = 2 * np.pi * c / twoP
                qc.cp(phase, qr_index, qr_data[qb])

        if dotranspile:
            qc = transpile(qc,
                           optimization_level=op_lvl,
                           basis_gates=bgate,
                           seed_transpiler=42)

        ##############################
        return qc.to_gate(label=" [+" + str(const) + "] ") if to_gate else qc

    # penalty part
    def const_adder(data_qubits: int,
                    const: int,
                    to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(1)
        qr_data = QuantumRegister(data_qubits, "data")
        qc = QuantumCircuit(qr_index, qr_data)

        ##############################
        ### Phase Rotation ###
        # Use `subroutine_add_const`
        qc.append(subroutine_add_const(data_qubits, const, to_gate),
                  [qr_index[:]] + qr_data[:])

        ##############################

        return qc.to_gate(label=" [ +" + str(const) + "] ") if to_gate else qc

    # penalty part
    def cost_calculation(index_qubits: int,
                         data_qubits: int,
                         list1: list,
                         list2: list,
                         to_gate=True) -> Union[Gate, QuantumCircuit]:

        qr_index = QuantumRegister(index_qubits, "index")
        qr_data = QuantumRegister(data_qubits, "data")
        qc = QuantumCircuit(qr_index, qr_data)

        # use diff calculated before

        diff_str_inv = bin(target_min_c + diff)[2:].zfill(data_qubits)[::-1]

        for ii in range(data_qubits):
            if diff_str_inv[ii] == '1':
                qc.x(qr_data[ii])

        if C2[0] > C1[0]:  #just to make sure
            first_val_diff = C2[0] - C1[0] + target_min_c + diff
        else:
            first_val_diff = C1[0] - C2[0] + target_min_c + diff

        first_val_diff_str = bin(first_val_diff)[2:].zfill(data_qubits)[::-1]

        for ii in range(data_qubits):
            if diff_str_inv[ii] != first_val_diff_str[ii]:
                qc.cx(qr_index[0], qr_data[ii])
        if dotranspile:
            qc = transpile(qc,
                        basis_gates=bgate,
                        optimization_level=op_lvl,
                        seed_transpiler=42)


        # no transpile a_d = 1
        qft = QFT(data_qubits,
                  approximation_degree=0,
                  do_swaps=False,
                  inverse=False,
                  insert_barriers=False,
                  name=None).decompose()

        if dotranspile:
            qft = transpile(qft,
                            basis_gates=bgate,
                            optimization_level=op_lvl,
                            seed_transpiler=42)
        qc.append(qft, qr_data)

        for i, (val1, val2) in enumerate(zip(list1[1:], list2[1:])):

            ##############################
            ### Add val2 using const_adder controlled by i-th index register (set to 1) ###
            # Provide your code here
            if val2 > val1:
                ctrl_adder_gate2 = const_adder(data_qubits, val2 - val1, True)
                qc.append(ctrl_adder_gate2, [qr_index[i + 1]] + qr_data[:])
            ##############################
        if dotranspile:
            qc = transpile(qc,
                           basis_gates=bgate,
                           optimization_level=op_lvl,
                           seed_transpiler=42)
        qfti = QFT(data_qubits,
                   approximation_degree=0,
                   do_swaps=False,
                   inverse=True).decompose()

        if dotranspile:
            qfti = transpile(qfti,
                             basis_gates=bgate,
                             optimization_level=2,
                             seed_transpiler=42)
        qc.append(qfti, qr_data)
        qc = qc.decompose()
        if dotranspile:
            qc = transpile(qc,
                           basis_gates=bgate,
                           optimization_level=op_lvl,
                           seed_transpiler=42)

        return qc.to_gate(label=" Cost Calculation ") if to_gate else qc

    # penalty part
    def constraint_testing(data_qubits: int,
                           C_max: int,
                           to_gate=True) -> Union[Gate, QuantumCircuit]:

        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_data, qr_f)

        ##############################
        ### Set the flag register for indices with costs larger than C_max ###
        # Provide your code here
        # do what the paper suggested
        if ultra:
            qc.cx(qr_data[-1], qr_f)
        else:
            qc.x(qr_f)
            qc.x(qr_data[w_bits:])
            qc.mct(qr_data[w_bits:], qr_f)
            qc.x(qr_data[w_bits:])
        if dotranspile:
            qc = transpile(qc,
                           optimization_level=op_lvl,
                           basis_gates=bgate,
                           seed_transpiler=42)
        ##############################

        return qc.to_gate(label=" Constraint Testing ") if to_gate else qc

    # penalty part
    def penalty_dephasing(data_qubits: int,
                          alpha: float,
                          gamma: float,
                          to_gate=True) -> Union[Gate, QuantumCircuit]:

        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_data, qr_f)

        ##############################
        ### Phase Rotation ###
        # Provide your code here
        for ii in range(data_qubits):
            qc.cp(alpha * gamma * 2**(ii), qr_f, qr_data[ii])

        qc.p(-alpha * gamma * (2**w_bits - adjust), qr_f)
        if dotranspile:
            qc = transpile(qc,
                           optimization_level=op_lvl,
                           basis_gates=bgate,
                           seed_transpiler=42)
        ##############################
        return qc.to_gate(label=" Penalty Dephasing ") if to_gate else qc

    # penalty part
    def reinitialization(index_qubits: int,
                         data_qubits: int,
                         C1: list,
                         C2: list,
                         C_max: int,
                         to_gate=True) -> Union[Gate, QuantumCircuit]:

        qr_index = QuantumRegister(index_qubits, "index")
        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_index, qr_data, qr_f)

        ##############################
        ### Reinitialization Circuit ###
        # Provide your code here

        ctt_inv = constraint_testing(data_qubits, C_max,
                                     to_gate=True).inverse()

        qc.append(ctt_inv, qr_data[:] + qr_f[:])

        cost_inv = cost_calculation(index_qubits,
                                    data_qubits,
                                    C1,
                                    C2,
                                    to_gate=True).inverse()

        qc.append(cost_inv, qr_index[:] + qr_data[:])

        if dotranspile:
            qc = transpile(qc,
                           optimization_level=op_lvl,
                           basis_gates=bgate,
                           seed_transpiler=42)

        #print_score(qc)
        ##############################

        return qc.to_gate(label=" Reinitialization ") if to_gate else qc

    ### Mixing Operator ###
    def mixing_operator(index_qubits: int,
                        beta: float,
                        to_gate=True) -> Union[Gate, QuantumCircuit]:

        qr_index = QuantumRegister(index_qubits, "index")
        qc = QuantumCircuit(qr_index)

        ##############################
        ### Mixing Operator ###
        # Provide your code here

        qc.rx(2 * beta, qr_index)
        if dotranspile:
            qc = transpile(qc,
                           optimization_level=op_lvl,
                           basis_gates=bgate,
                           seed_transpiler=42)
        ##############################
        return qc.to_gate(label=" Mixing Operator ") if to_gate else qc

    qr_index = QuantumRegister(index_qubits, "index")  # index register
    qr_data = QuantumRegister(data_qubits, "data")  # data register
    qr_f = QuantumRegister(1, "flag")  # flag register
    if dotest < 5:
        cr_index = ClassicalRegister(
            index_qubits + 1 + data_qubits, "c_index"
        )  # classical register storing the measurement result of index register
        qc = QuantumCircuit(qr_index, qr_data, qr_f, cr_index)

    else:
        cr_index = ClassicalRegister(
            index_qubits, "c_index"
        )  # classical register storing the measurement result of index register
        qc = QuantumCircuit(qr_index, qr_data, qr_f, cr_index)

    ### initialize the index register with uniform superposition state ###
    qc.h(qr_index)

    ### DO NOT CHANGE THE CODE BELOW
    # Notice, I just formatted the rest
    p = 5
    alpha = 1
    for i in range(p):

        ### set fixed parameters for each round ###
        beta = 1 - (i + 1) / p
        gamma = (i + 1) / p

        ### return part ###
        qc.append(phase_return(index_qubits, gamma, L1, L2), qr_index)

        ### step 1: cost calculation ###
        qc.append(cost_calculation(index_qubits, data_qubits, C1, C2),
                  qr_index[:] + qr_data[:])

        ### step 2: Constraint testing ###
        qc.append(constraint_testing(data_qubits, C_max), qr_data[:] + qr_f[:])

        ### step 3: penalty dephasing ###
        qc.append(penalty_dephasing(data_qubits, alpha, gamma),
                  qr_data[:] + qr_f[:])

        ### step 4: reinitialization ###
        qc.append(reinitialization(index_qubits, data_qubits, C1, C2, C_max),
                  qr_index[:] + qr_data[:] + qr_f[:])
                  
        if dotest == i:
            qc.measure(qr_index,
                       cr_index[index_qubits + data_qubits:data_qubits:-1])
            qc.measure(qr_data, cr_index[data_qubits:0:-1])
            qc.measure(qr_f, cr_index[0])
            return qc
        ### mixing operator ###
        qc.append(mixing_operator(index_qubits, beta), qr_index)

    ### measure the index ###
    ### since the default measurement outcome is shown in big endian, it is necessary to reverse the classical bits in order to unify the endian ###
    qc.measure(qr_index, cr_index[::-1])

    return qc

In [9]:
simulator = Aer.get_backend("qasm_simulator")

In [31]:
dotest = 455
qcs, job = prepare_ex4c(solver_function)
pass_or_not, score = grade(qcs, job.result())
print(pass_or_not)
print(score)
# for ii in range(24):
#      print(ii)
#      reinit_correct(job.result().get_counts(ii),**myps[ii])
#      #add_correct(job.result().get_counts(ii),**myps[ii])
#      #both_correct(job.result().get_counts(ii),**myps[ii])

run job
created jobs ( 3.9580557346343994 s)
Precision for instance 1 : 1.0
depth: 1085
num_ops: OrderedDict([('rz', 1820), ('cx', 1410), ('sx', 250), ('measure', 11)])
Score: 70420

Precision for instance 2 : 1.0
depth: 1087
num_ops: OrderedDict([('rz', 1835), ('cx', 1460), ('sx', 270), ('measure', 11)])
Score: 71055

Precision for instance 3 : 1.0
depth: 1117
num_ops: OrderedDict([('rz', 1760), ('cx', 1370), ('sx', 250), ('measure', 11)])
Score: 71560

Precision for instance 4 : 1.0
depth: 1085
num_ops: OrderedDict([('rz', 1800), ('cx', 1390), ('sx', 230), ('measure', 11)])
Score: 70180

Precision for instance 5 : 1.0
depth: 1127
num_ops: OrderedDict([('rz', 1880), ('cx', 1500), ('sx', 270), ('measure', 11)])
Score: 73500

Precision for instance 6 : 1.0
depth: 1087
num_ops: OrderedDict([('rz', 1780), ('cx', 1400), ('sx', 270), ('measure', 11)])
Score: 70400

Precision for instance 7 : 1.0
depth: 1117
num_ops: OrderedDict([('rz', 1820), ('cx', 1430), ('sx', 270), ('measure', 11)])
Sco

In [32]:
for ii in range(24):
    #print(ii)
    plot_hist(job.result().get_counts(ii),**myps[ii])

In [26]:
do_submit = True
sim = True
dotranspile = False
if not sim:
    real_res = job.result()

precisions = [0] * 8
best_percent = [0] * 8
for i in range(8):
    print(i)
    for kk in range(1):
        if sim == True:
            testqc = solver_function(**instance_examples[i])
            print_score(testqc)
            testqct = transpile(testqc, backend=simulator)
            simjob = simulator.run(testqct, shots=512)
            simres = simjob.result().get_counts()
        else:
            simres = real_res.get_counts(i)
        precisions[i], best_percent[i] = test_res(simres, **instance_examples[i])
        print(precisions[i],best_percent[i])
print(sum(precisions[:4])/4)
print(sum(precisions[4:])/4)
if sum(precisions[:4]) / 4 < 0.80:
    do_submit = False


0
70064
0.952255908331344 1.0
1
68045
0.9395949545203277 1.0
2
69344
0.9513385293120976 1.0
3
69622
0.9287352367847724 1.0
4
66624
0.913510101010101 1.0
5
69610
0.9496211251435132 1.0
6
70490
0.9331694756554307 1.0
7
67057
0.9574762658227848 1.0
0.9429811572371354
0.9384442419079575


In [48]:
for instance in instance_examples:
    print(perf_ans_calc(**instance),instance['C_max'])

('11001101011', 36, 23) 36
('10101011001', 35, 21) 35
('11010110010', 39, 21) 39
('10111110000', 37, 21) 37
('10001010111', 34, 19) 34
('10100100111', 41, 26) 41
('10101101100', 40, 25) 40
('00111001110', 38, 33) 38


In [92]:
print_score(testqc)
# 106375

70490


70490

In [93]:
# Execute your circuit with following prepare_ex4c() function.
# The prepare_ex4c() function works like the execute() function with only QuantumCircuit as an argument.
from qc_grader import prepare_ex4c
from qc_grader.grade import grade_job

print(do_submit)
do_submit = True
if do_submit:
    #instance_examples = []
    sim = False
    job = prepare_ex4c(solver_function)
    


True
Running "solver_function" (1/8)... 
Running "solver_function" (2/8)... 
Running "solver_function" (3/8)... 
Running "solver_function" (4/8)... 
Running "solver_function" (5/8)... 
Running "solver_function" (6/8)... 
Running "solver_function" (7/8)... 
Running "solver_function" (8/8)... 
Starting experiments. Please wait...
You may monitor the job (id: 6185357adafab3a33794299e) status and proceed to grading when it successfully completes.


In [94]:
from time import sleep
sleep(30)
grade_job(job, '4c')

Grading your answer for 4c. Please wait...

Congratulations 🎉! Your answer is correct.
Your score is 277075.


(True, 277075)

In [43]:
# Check your answer and submit using the following code
from qc_grader import grade_ex4c
grade_ex4c(job)

Submitting your answer for 4c. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.
Your score is 273998.


### References
1. Edward Farhi and Jeffrey Goldstone and Sam Gutmann (2014). A Quantum Approximate Optimization Algorithm. (https://arxiv.org/abs/1411.4028)
2. Grand'rive, Pierre & Hullo, Jean-Francois (2019). Knapsack Problem variants of QAOA for battery revenue optimisation.  (https://arxiv.org/abs/1908.02210)
3. V. Vedral, A. Barenco, A. Ekert (1995). Quantum Networks for Elementary Arithmetic Operations. (https://arxiv.org/abs/quant-ph/9511018)
4. Steven A. Cuccaro, Thomas G. Draper, Samuel A. Kutin, David Petrie Moulton (2004). A new quantum ripple-carry addition circuit. (https://arxiv.org/abs/quant-ph/0410184)
5. Thomas G. Draper (2000). Addition on a Quantum Computer (https://arxiv.org/abs/quant-ph/0008033)
6. Lidia Ruiz-Perez, Juan Carlos Garcia-Escartin (2014). Quantum arithmetic with the Quantum Fourier Transform. (https://arxiv.org/abs/1411.5949)

## Additional information

**Created by:** Bo Yang, Hyungseok Chang, Sitong Liu, Kifumi Numata

**Version:** 1.0.1